# Analysing MusicNet exepriments

In [ ]:
import os
import re

import json
import numpy as np
import torch

from cplxpaper.auto import auto
from cplxpaper.auto.utils import load_snapshot

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

Borrowed from mnist-like

In [ ]:
from cplxpaper.auto.parameter_grid import get_params

def get_details(self):
    out = dict()
    for key in self:
        value = self[key]
        if isinstance(value, (dict, list, tuple)):
            if isinstance(value, (list, tuple)):
                value = {f"[{i}]": v for i, v in enumerate(value)}
                nested = get_details(value).items()
                out.update((key + k, val) for k, val in nested)

            elif isinstance(value, dict):
                nested = get_details(value).items()
                out.update((key + '__' + k, val) for k, val in nested)

            continue

        out[key] = value

    return out

A useful feed wrapper with a progress bar

In [ ]:
import tqdm
from functools import wraps

from cplxpaper.auto.feeds import BaseFeedWrapper

class TqdmFeed(BaseFeedWrapper):
    @wraps(tqdm.tqdm.__init__)
    def __init__(self, feed, **kwargs):
        super().__init__(feed)
        self.kwargs = kwargs

    def __iter__(self):
        with tqdm.tqdm(self.feed, **self.kwargs) as feed:
            yield from feed

Load feeds from a recent manifest

In [ ]:
def load_feeds(options, devtype):
    return auto.get_feeds(auto.get_datasets(options["datasets"]),
                          devtype, options["features"], options["feeds"])

Legacy version of the feed loader.

In [ ]:
def load_feeds_legacy(optoons, devtype):
    # construct datasets from pieces
    datasets = {}
    for name, param in options["dataset_sources"].items():
        datasets[name] = auto.get_instance(
            **options["dataset"], **param)

    # get the designated batch collation function
    collate_fn = auto.get_instance(**options["features"])

    # make instances of batch feeds
    recipe = auto.param_apply_map(
        options["feeds"], dataset=datasets.__getitem__)

    feeds = {}
    for name, par in recipe.items():
        par = auto.param_defaults(par, n_batches=-1, pin_memory=True,
                                  cls=str(torch.utils.data.DataLoader))

        max_iter = par.pop("n_batches")
        feed = get_instance(**par, collate_fn=collate_fn)
        feeds[name] = auto.wrap_feed(feed, max_iter=max_iter, **devtype)

    return feeds

Load model from the manifest

In [ ]:
def load_model(snapshot):
    options, stage, config = snapshot["options"], *snapshot["stage"]
    model = auto.get_model(options["model"], **config["model"])
    model.load_state_dict(snapshot["model"])
    
    return model

In [ ]:
from cplxpaper.auto.utils import load_snapshot

def from_snapshots(*snapshots):
    results, options = {}, {}
    for snapshot in sorted(snapshots):
        name = os.path.basename(snapshot)
        snapshot = load_snapshot(snapshot)

        options = snapshot['options']
        stage, settings = snapshot['stage']

        results[name] = stage, snapshot['performance']

    return results, options

In [ ]:
def score_snapshot(scorers, snapshot, **devtype):
    snapshot = load_snapshot(snapshot)
    model = load_model(snapshot).to(**devtype)

    # sad that the api was changed during the run of the experiment.
    if "fft-shifted" in snapshot["options"]["features"]["kind"]:
        scorers = {name[8:]: scorer for name, scorer in scorers.items()
                   if name.startswith("shifted-")}
    else:
        scorers = {name: scorer for name, scorer in scorers.items()
                   if not name.startswith("shifted-")}
    
    scores = {name: scorer(model.eval()) for name, scorer in scorers.items()}

    stored = snapshot['performance']
    common = stored.keys() & scores.keys()
    if stored:
        assert common
    for scorer_name in common:
        original = stored[scorer_name]
        result = scores[scorer_name]
        for name in result.keys() & original.keys():
            l, r = result[name], original[name]
            assert type(l) == type(r)
            if isinstance(l, np.ndarray):
                assert np.allclose(l, r, equal_nan=True, atol=1e-3, rtol=2e-3)

    return scores, snapshot["options"]

def score_snapshots(scorers, *snapshots, **devtype):
    scores, options = {}, {}
    for snapshot in snapshots:
        score, options = score_snapshot(scorers, snapshot, **devtype)
        scores[os.path.basename(snapshot)] = score

    return scores, options

In [ ]:
import pickle

def score_experiment(scorers, experiment, cache="cache.pk", **devtype):
    if isinstance(cache, str):
        cache = os.path.join(folder, cache)
#         if os.path.isfile(cache):
#             os.unlink(cache)

    assert cache is None or isinstance(cache, str)

    snapshots = []
    experiment, _, filenames = next(os.walk(experiment))
    for filename in filenames:
        match = re.match(r"^(?!\.).*?\.gz$", filename)
        if match is None or not os.path.isfile(os.path.join(experiment, filename)):
            continue

        snapshots.append(filename)

    # load scorer results from the snapshots or from cache
    scores, options = {}, {}
    if cache is not None and os.path.exists(cache):
        with open(cache, "rb") as fin:
            scores, options = pickle.load(fin)

    # reload from originals if anything is missing (use SHA-digest?)
    if any(s not in scores for s in snapshots):
        snapshots = [os.path.join(experiment, s) for s in snapshots]

        scores, options = score_snapshots(scorers, *snapshots, **devtype)
        if cache is not None:
            with open(cache, "wb") as fout:
                pickle.dump((scores, options), fout)

    return scores, options

<br>

Due to lib's version mismatch some of the runs of the musicnet
experiment have missing pieces of performance scoring.

Therefore we setup a separate pipeline to perform evaluation

In [ ]:
pipeline = {
    "datasets": {
        "musicnet-test-128": {
          "cls": "<class 'cplxpaper.musicnet.dataset.MusicNetRAM'>",
          "filename": "/home/ivan.nazarov/Github/complex_paper/experiments/musicnet/data/musicnet_11khz_test.h5",
          "window": 4096,
          "stride": 128
        }
    },
    "features": {
        "cls": "<class 'cplxpaper.auto.feeds.FeedFourierFeatures'>",
        "signal_ndim": 1,
        "cplx": True,
        "shift": False
    },
    "feeds": {
        "test-256": {
            "cls": "<class 'torch.utils.data.dataloader.DataLoader'>",
            "dataset": "musicnet-test-128",
            "batch_size": 256,
            "pin_memory": True,
            "shuffle": False,
            "n_batches": -1
        }
    },
    "scorers": {
        "test_256": {
            "cls": "<class 'cplxpaper.musicnet.performance.MusicNetBasePerformance'>",
            "feed": "test-256",
            "curves": True
        }
    }
}

In [ ]:
pipeline_shifted = {
    "datasets": {
        "musicnet-test-128": {
          "cls": "<class 'cplxpaper.musicnet.dataset.MusicNetRAM'>",
          "filename": "/home/ivan.nazarov/Github/complex_paper/experiments/musicnet/data/musicnet_11khz_test.h5",
          "window": 4096,
          "stride": 128
        }
    },
    "features": {
        "cls": "<class 'cplxpaper.auto.feeds.FeedFourierFeatures'>",
        "signal_ndim": 1,
        "cplx": True,
        "shift": True
    },
    "feeds": {
        "test-256": {
            "cls": "<class 'torch.utils.data.dataloader.DataLoader'>",
            "dataset": "musicnet-test-128",
            "batch_size": 256,
            "pin_memory": True,
            "shuffle": False,
            "n_batches": -1
        }
    },
    "scorers": {
        "shifted-test_256": {
            "cls": "<class 'cplxpaper.musicnet.performance.MusicNetBasePerformance'>",
            "feed": "test-256",
            "curves": True
        }
    }
}

Create a default scorer from the pipeline manifest just defined.

In [ ]:
devtype = dict(device=torch.device("cuda:3"))

feeds = load_feeds(pipeline, devtype)
scorers = auto.get_scorers({
    k: TqdmFeed(f) for k, f in feeds.items()
}, pipeline["scorers"])

feeds_shifted = load_feeds(pipeline_shifted, devtype)
scorers.update(
    auto.get_scorers({
        k: TqdmFeed(f) for k, f in feeds_shifted.items()
    }, pipeline_shifted["scorers"])
)

Gather oll MusicNet experiments.

In [ ]:
rootdir, dirnames, manifests = next(os.walk("./runs/grid_trabelsi"))

experiments = [m for m, e in map(os.path.splitext, manifests)
               if e == ".json" and os.path.isdir(os.path.join(rootdir, m))]

In [ ]:
scores = {}
for experiment in experiments:
    folder = os.path.join(rootdir, experiment)
    scores[experiment] = score_experiment(scorers, folder, **devtype)

In [ ]:
from collections import defaultdict

ignore = {"__name__", "__timestamp__", "__version__", "device"}
grid_options, results = defaultdict(set), []
for experiment, (score, options) in scores.items():
    match = re.match(r"^.*?\[(\d+)\]-(\d+)$", experiment)
    replication, exp_no = map(int, match.groups())

    score = {k: v["test_256"] for k, v in score.items()}

    flat = get_details(options)
    for k, v in flat.items():
        if k not in ignore:
            grid_options[k].add(v)
    
    res = {}
    for snapshot, result in score.items():
        match = re.match(r"^\d+-([\w-]+)\s.*$", snapshot)
        stage, = match.groups()

        n_zer, n_par = map(sum, zip(*result["sparsity"].values()))
        res[stage] = {
            "n_zer": n_zer,
            "n_par": n_par,
            "ap-score": result["pooled_average_precision"],
#             "ap-curve": []
        }
    results.append(((exp_no, replication), res, flat))

grid = [k for k, v in grid_options.items() if len(v) > 1]

In [ ]:
from cplxpaper.auto.utils import get_class

cls = next(iter(grid_options["dataset__cls"]))

datasetname = get_class(cls).__name__

In [ ]:
import pandas as pd
keys, scores, manifests = zip(*results)

params = [{k: opt.get(k, None) for k in grid} for opt in manifests]
params = pd.concat(dict(zip(keys, map(pd.Series, params))),
                   axis=1, names=["expno", "replication"])

scores = pd.concat(dict(zip(keys, map(pd.DataFrame, scores))),
                   axis=1, names=["expno", "replication"])

In [ ]:
params = params.T.sort_index(axis=0)
scores = scores.stack().T.swaplevel(axis=1)
scores = scores.sort_index(axis=1).sort_index(axis=0)
scores.columns = scores.columns.to_flat_index().map('-'.join)

In [ ]:
df = params.join(scores).reset_index()
main_grid = [g for g in grid if not g.endswith('__kl_div')]
df = df.set_index([*grid, "expno", "replication"], append=False, drop=True).sort_index(0)

In [ ]:
summary = {}
for k, g in df.groupby(axis=0, level=main_grid):
    g = g.loc[k]

    ap_before = g["dense-ap-score"].mean(), g["dense-ap-score"].std()

    ap_after, n_par, n_zer = g["fine-tune-ap-score"], g["sparsify-n_par"], g["sparsify-n_zer"]
    curve = pd.concat([n_zer / n_par, ap_after], axis=1)
#     curve = curve.mean(level=0).to_numpy()
    curve = curve.to_numpy()
    order = curve[:, 0].argsort()

    summary[k] = ap_before, curve[order]

In [ ]:
from matplotlib.ticker import FormatStrFormatter, FuncFormatter
import time
dttm = time.strftime("%Y%m%d-%H%M%S")

figurename = os.path.basename(rootdir)

filename = os.path.join(
    os.path.dirname(os.path.abspath(rootdir)),
    f"{figurename} {dttm}.pdf")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 5))
fig.patch.set_alpha(1.0)

for name, (dense, curve) in summary.items():
    m, s = dense
    spr, acc = curve.T
    pts = ax.scatter(1 - spr, acc, label=name, s=15)
    color = pts.get_facecolor()[0]
#     pts, = ax.plot(1 - spr, acc, label=name)
#     color = pts.get_color()
    ax.axhspan(m - 1.96 * s, m + 1.96 * s, alpha=0.1, color=color)

ax.legend(ncol=2)
ax.set_title(datasetname)
ax.set_ylabel("accuracy")

ax.set_xlabel("% nonzero")
ax.set_xscale("log")
ax.set_xlim(1e-2, 1.1)

ax.xaxis.set_major_formatter(FuncFormatter(lambda x, p: f"{x:.1%}"))

fig.savefig(filename, dpi=300, transparent=False)

plt.show()

In [ ]:
assert False

In [ ]:
experiment

Load a mode from this specified snapshot

In [ ]:
folder = """./runs/grid_trabelsi/musicnet[000]-021/"""
!ls {folder}

In [ ]:
# snapshot = load_snapshot(os.path.join(folder, "1-sparsify 20191228-023901.gz"))
snapshot = load_snapshot(os.path.join(folder, "0-dense StopIteration 20191228-110511.gz"))
# model = load_model(snapshot).to(**devtype)

In [ ]:
snapshot["early_history"]

In [ ]:
original = snapshot["performance"]["test_256"]

In [ ]:
original

In [ ]:
result = scorers['shifted-test_256'](model)

In [ ]:
for name in result.keys() & original.keys():
    l, r = result[name], original[name]
    assert type(l) == type(r)
    if isinstance(l, np.ndarray):
        assert np.allclose(l, r, equal_nan=True, atol=1e-3, rtol=2e-3)

In [ ]:
l - r